# MAIA - Machine Learning No Supervisado
## Microproyecto 1: Generación de paleta de colores utilizando MLNS

### Estudiantes:
- Claudia Agudelo
- Felipe Flórez

### **Descripción de la aplicación:**

Una paleta de colores es un conjunto de tonos utilizados para crear armonía visual y transmitir emociones en diversas formas de arte. Ante la creciente demanda de herramientas que asistan a creadores en la selección de colores, una aplicación que genere paletas a partir de imágenes sería muy útil. El reto es desarrollar un método automatizado que no solo identifique colores dominantes, sino que también cree combinaciones visualmente atractivas. Esto optimizaría el proceso creativo, garantizaría consistencia visual, facilitaría el diseño para principiantes y mejoraría la comunicación visual. Una posible solución es aplicar técnicas de machine learning no supervisado para generar paletas basadas en la distribución de colores en imágenes, con aplicaciones en diversos campos como marketing, arte y estudios ambientales.

#### **Objetivo**

Desarrollar un método, basado en técnicas de agrupación, que permita extraer los tonos de una imagen y generar un muestrario de los colores presentes en esta.

#### **Conjunto de datos**

Los datos están asociados con imágenes de obras de arte. Pueden ser descargados a partir de este [WikiArt](https://www.kaggle.com/datasets/steubk/wikiart/data)

#### **Actividades a realizar**

1. Recopilación de las imágenes a partir del repositorio. La idea es seleccionar un conjunto diverso de muestras en diferentes estilos artísticos.

2. Preparación de las imágenes para el entrenamiento y prueba del modelo. Para este paso construir un pipeline que integre las transformaciones que se consideren adecuadas.

3. Desarrollo del modelo de agrupación para identificar los colores presentes en las imágenes.

4. Creación de un modelo que transforme los grupos de colores identificados en un muestrario representativo. Adicionalmente, se debería mostrar la distribución de los colores de la imagen en un espacio de dos dimensiones.


* El algoritmo de agrupación a utilizar queda a consideración de cada grupo, pero es importante justificar la elección.

* El número de colores de una paleta generada a partir de una imagen debería estar entre 5 y 7.

* Para la visualización de la distribución de colores seleccionar un método como t-SNE.

## Carga de los datos

In [ ]:
# Optimizador de ejecución con GPU
!pip install numba

In [ ]:
#Uso de temporizador para ejecución
import time

In [ ]:
# Librerías a utilizar
from numba import cuda
import os
import random
import shutil
from PIL import Image
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Procesamiento de datos
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Modelos de Machine Learning No Supervisado
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

# Reducción de dimensionalidad
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Métricas de evaluación
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, adjusted_rand_score
from sklearn.model_selection import GridSearchCV

# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

Pasos:

1. Se define una ruta a una carpeta (main_folder) que contiene subcarpetas con imágenes.
2. Se selecciona aleatoriamente una subcarpeta y luego se copian un máximo de 100 imágenes de esa carpeta a una nueva carpeta llamada imagenes.
   
El objetivo es almacenar las imágenes en un directorio específico para su posterior procesamiento.

In [ ]:
#Ruta de la carpeta de las imagenes
main_folder = "archive"

#Obtener las carpetas de estilos, filtrando solo las que son directorios
folders = [folder for folder in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, folder))]

#Seleccionar 10 carpetas aleatorias
selected_folders = random.sample(folders, 1)

#Crear una carpeta para almacenar las imágenes seleccionadas en el escritorio
output_folder = "imagenes"
os.makedirs(output_folder, exist_ok=True)

#Seleccionar imágenes aleatorias de cada carpeta seleccionada
for folder in selected_folders:
    folder_path = os.path.join(main_folder, folder)
    images = os.listdir(folder_path)

    #Verificar si hay menos de 100 imágenes
    num_images_to_select = min(100, len(images))
    selected_images = random.sample(images, num_images_to_select)

    for image in selected_images:
        src_path = os.path.join(folder_path, image)
        dst_path = os.path.join(output_folder, image)
        shutil.copy(src_path, dst_path)

## Carga de imagenes almacenadas

Creamos una función para cargar imágenes desde un directorio al espacio de trabajo. Luego, visualizamos una imagen de muestra y analizamos sus características, incluyendo: la visualización de la imagen, los valores mínimo y máximo de los píxeles, sus dimensiones (número de píxeles por dimensión), el número de canales de color, y el tipo de codificación de los píxeles.

Pasos:
1. Definimos la clase Image_loader, la cual permite cargar imágenes de una carpeta específica y visualizar las imágenes seleccionadas.
2. Ahora hacemos uso del método loader, ya que este carga las imágenes basadas en índices proporcionados, y plotter permite mostrar la imagen y analizar sus propiedades (dimensiones, normalización, número de canales, etc.).

In [ ]:
# Constructor para cargar imagenes
class Image_loader:
    def __init__(self, folder_path, image_indices = (0,1)):
        self.folder_path = folder_path
        self.images = self.loader(image_indices)

    # Lector de imágenes
    def loader(self, image_indices = (0,1)):
        images = []
        image_folder = self.folder_path
        image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for img_path in image_paths[image_indices[0]: image_indices[1]]:
            try:
                image = Image.open(img_path)
                images.append(np.array(image))
            except Exception as e:
                print(f"Error al cargar la imagen {img_path}: {e}")
                images.append(None)
        return images

    # Graficador de imágenes
    def plotter(self, index_image=0):
        if index_image < 0 or index_image >= len(self.images):
            print(f"Índice fuera de rango. Hay {len(self.images)} imágenes disponibles.")
        else:
            try:
                selected_image = self.images[index_image]
                if selected_image is not None:
                    # Mostrar la imagen
                    plt.imshow(selected_image)
                    plt.axis('off')
                    plt.show()

                    # Dimensiones de la imagen
                    height, width = selected_image.shape[:2]
                    print(f"Dimensiones: {height}x{width}")

                else:
                    print(f"La imagen en el índice {index_image} no es válida.")

            except Exception as e:
                print(f"Error al graficar la imagen: {e}")

Pasos:

1. Ahora se utiliza la clase Image_loader previamente definida para cargar imágenes de la carpeta imagenes.
2. Luego, seleccionamos un índice aleatorio de entre las imágenes cargadas y se visualiza la imagen correspondiente utilizando el método plotter.

Para ilustrar la manera en la que opera el anterior constructor, cargamos una serie de imágenes y procedemos a visualizar una imagen de ejemplo de nuestro interés.

In [ ]:
import random

# Ruta de la carpeta de las imágenes
image_folder = 'imagenes'

# Imagen de ejemplo
indices_imagenes = (0, 500)
loader = Image_loader(folder_path=image_folder, image_indices=indices_imagenes)

# Generar un valor aleatorio distinto de 0
random_index = random.choice([i for i in range(indices_imagenes[0], indices_imagenes[1] + 1) if i != 0])

# Usar el valor aleatorio en loader.plotter
loader.plotter(index_image=random_index)

## Preparación de datos y creación del Pipeline

En la preparación de datos, se crea un pipeline con los siguientes pasos:
- **Resizer**: Redimensiona todas las imágenes a un tamaño fijo de (128,128) píxeles.
- **Normalizer**: Escala los valores de los píxeles al rango [0,1].
- **Convertidor de color**: Convierte las imágenes en blanco y negro al formato RGB de tres canales.
- **Squasher**: Aplana las imágenes en un array $(l,3)$, donde $l = m \times n$, siendo $m$ y $n$ el alto y ancho de la imagen, respectivamente.

Pasos:

1. Ahora haremos uso de la clase Resizer para redimensionar las imágenes a un tamaño especificado, que por defecto es 128x128 píxeles.
2. Luego utilizaremos el método transform para redimensionar cada imagen y devolverla como un arreglo NumPy, lo que será útil para estandarizar todas las imágenes antes de aplicar algún análisis o procesamiento adicional.

In [ ]:
# Transformador para redimensionar imágenes
class Resizer(BaseEstimator, TransformerMixin):
    def __init__(self, size=(128, 128)):
        self.size = size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Usamos map para aplicar el resize a cada imagen en X
        return np.array([self._resize_image(img_path) for img_path in X])

    def _resize_image(self, img_path):
        try:
            image = Image.open(img_path).resize(self.size)
            return np.array(image)
        except Exception as e:
            print(f"Error de redimensionamiento de la imagen {img_path}: {e}")
            return None

Pasos:

1. Ahora usamos la clase Normalizer para normalizar los valores de los píxeles de las imágenes dividiéndolos por 255.0, lo que los lleva a un rango de [0, 1].

La normalización de los datos es importante cuando se utilizan algoritmos de aprendizaje automático, ya que ayuda a estabilizar el comportamiento de los modelos.

In [ ]:
# Transformador para normalizar imágenes
class Normalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Aseguramos que X sea un array de NumPy para realizar la normalización de forma eficiente
        X = np.asarray(X, dtype=np.float32)
        return X / 255.0

Pasos: 

1. Ahora usamos la clase InverseNormalizer para realizar el proceso inverso de la normalización, devolviendo los valores de los píxeles al rango original [0, 255]. Esto puede ser útil si es necesario visualizar las imágenes o restaurarlas después de realizar algún análisis o procesamiento.

In [ ]:
# Transformador inverso para denormalizar imágenes
class InverseNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Aseguramos que X sea un array de NumPy para realizar la denormalización de forma eficiente
        X = np.asarray(X, dtype=np.float32)
        return (X * 255).astype(np.uint8)

Pasos:

1. Ahora utilizamos el transformador ColorConverter, el cual asegura que todas las imágenes tengan tres canales de color (Rojo, Verde, Azul). Si alguna imagen es en blanco y negro (2D), se duplica la imagen en los tres canales. Esto es importante para asegurarse de que todas las imágenes estén en un formato RGB adecuado antes de realizar el análisis.

In [ ]:
# Transformador para los canales R, B, G
class ColorConverter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Aseguramos que X sea un array de NumPy para realizar las operaciones de forma eficiente
        X = np.asarray(X)

        # Usamos list comprehension para simplificar la conversión
        return np.array([self._convert_image(image) for image in X])

    def _convert_image(self, image):
        if image is not None:
            # Si la imagen es en blanco y negro (2D), duplicamos los canales para hacerla RGB
            if len(image.shape) == 2:
                return np.stack((image,)*3, axis=-1)
        return image

Pasos:

1. Ahora implementamos el transformador Squasher, el cual toma una imagen y convierte su forma original (alto, ancho, 3) en una forma aplanada de (número total de píxeles, 3). En otras palabras, se requiere aplanar la imagen para que cada fila represente un píxel y cada columna un canal de color (R, G, B). Esta transformación es clave cuando se desea aplicar algoritmos de agrupación (como K-means) que trabajarán con las características de los píxeles.

In [ ]:
#Conversor a 3 dimensiones

class Squasher(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
        return self

    def transform(self,X):
        squashed_images = []
        for image in X:
            if image is not None:
                pixels = image.reshape(-1, 3)
                squashed_images.append(pixels)
            else:
                squashed_images.append(None)
        return np.array(squashed_images)

Pasos:

1. Ahora definimos un pipeline de preprocesamiento que aplica tres pasos a cada imagen:
1.1. Redimensionamiento a 128x128 píxeles.
1.2. Conversión de los canales de color para asegurar que todas las imágenes estén en formato RGB.
1.3. Normalización de los valores de los píxeles (de [0, 255] a [0, 1]).
   
Este pipeline se aplica a las primeras 500 imágenes almacenadas en la carpeta imagenes.

Veremos a continuación un ejemplo de la manera en la que los procedimientos Resizer,  Normalizer y ColorConverter funcionan. Generamos un pipeline de muestra y lo aplicamos a una imagen de ejemplo.

In [ ]:
# Aplicación del pipeline de preprocesamiento para una imagen

image_folder = "imagenes"

#Crear la lista de rutas de las imágenes
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]

#Definir el pipeline
preprocessing_pipeline = Pipeline([
    ('resize', Resizer(size=(128, 128))),
    ('color_convert', ColorConverter()),
    ('normalize', Normalizer()),
])

#Aplicar el pipeline
processed_images_array = preprocessing_pipeline.fit_transform(image_paths[0:500])

print(f"Se han procesado {len(processed_images_array)} imágenes.")

Pasos:

1. Ahora generamos un índice aleatorio dentro del rango de las imágenes preprocesadas, excluyendo el índice 0. Utilizamos random.choice para elegir un número al azar dentro de los índices válidos de las imágenes contenidas en processed_images_array, implementamos el processed_images_array ya que contiene las imágenes que han pasado por el proceso de redimensionamiento, normalización y posible conversión a formato RGB. Finalmente, utilizamos plt.imshow de la biblioteca Matplotlib se utiliza para visualizar la imagen seleccionada. Esta función toma una matriz (que representa la imagen) y la despliega en un gráfico.

In [ ]:
random_index = random.choice([i for i in range(1, len(processed_images_array))])

# Seleccionar la imagen con ese índice aleatorio
imagen_mat = processed_images_array[random_index]

# Graficar la imagen
plt.imshow(imagen_mat)
plt.show()

## Selección de los modelos de agrupación, sus hiperparámetros y sus métricas
En la presente sección, vamos a usar modelos diversos de agrupamiento para realizar la tarea de extracción de la paleta de colores característica, comparando la manera en como se comportan los modelos con los datos, validándolos con medidas de evaluación adecuadas para cada uno de ellos. Los modelos a utilizar son K-means, DBSCAN y Clustering Jerárquico.

Ahora iniciamos definiendo la clase de cluster de colores, estructurada de la siguiente manera:

Función __init__:
- method: Algoritmo de agrupación que se usará (por defecto, KMeans)
- scoring_metric: Métrica de evaluación que se usará para validar la calidad de los grupos. Para este caso tenemos silhouette, calinski_harabasz, y davies_bouldin
- param_grid: Conjunto de parámetros que se utilizarán para realizar una búsqueda de hiperparámetros con GridSearchCV
- squashed: Define si las imágenes ya han sido aplanadas o no. Si es False, el código aplanará las imágenes de forma predeterminada
- kwargs: Parámetros adicionales que se pasarán a los modelos de agrupación.

Función get_scoring_metric:
- silhouette_score: Mide qué tan similares son los objetos dentro de un grupo en comparación con otros grupos
- calinski_harabasz_score: Evalúa la dispersión entre grupos en relación con la dispersión dentro de los grupos
- davies_bouldin_score: Mide la relación entre la distancia entre los grupos y el tamaño de los grupos

Función fit:
- Método para entrenar un modelo de agrupación para cada imagen en el conjunto X (preprocesado).
- Consideramos que si squashed es True, la imagen ya está aplanada (2D), de lo contrario, se aplana aquí (convertida de una matriz 3D de colores a una matriz 2D donde cada fila representa un píxel RGB). Según el valor de method, se elige un algoritmo de agrupación: KMeans, DBSCAN o AgglomerativeClustering.
- GridSearchCV: Este proceso ajusta el modelo de agrupación mediante búsqueda de hiperparámetros, utilizando el conjunto de parámetros en param_grid y evaluando con la métrica seleccionada. Se selecciona el mejor modelo encontrado. Se guardan los modelos ajustados en self.models y las etiquetas de los clusters (grupos) en self.labels para cada imagen.

Función transform:
- Transformación de datos: El método transform() aplica el modelo entrenado a las imágenes. Para cada imagen, se agrupan los píxeles utilizando el modelo guardado en self.models.
- Extracción de colores: Para cada grupo (etiqueta label), se calcula el color promedio de los píxeles en ese grupo.
Se almacenan los resultados de agrupación de colores en cluster_results y cluster_image_data (que también incluye las posiciones de los píxeles agrupados).

In [ ]:
# Constructor de clusters
class ColorCluster(BaseEstimator, TransformerMixin):
    def __init__(self, param_grid, method='kmeans', scoring_metric='silhouette', squashed=False, **kwargs):
        self.method = method
        self.scoring_metric = scoring_metric
        self.param_grid = param_grid
        self.squashed = squashed
        self.kwargs = kwargs
        self.models = []
        self.labels = []

    # Métricas
    def get_scoring_metric(self):
        metrics = {
            'silhouette': silhouette_score,
            'calinski_harabasz': calinski_harabasz_score,
            'davies_bouldin': davies_bouldin_score
        }
        return metrics.get(self.scoring_metric, silhouette_score)

    # Función auxiliar para seleccionar modelo de clustering
    def _get_model(self):
        models = {
            'kmeans': KMeans(**self.kwargs),
            'density': DBSCAN(**self.kwargs),
            'hierarchical': AgglomerativeClustering(**self.kwargs)
        }
        if self.method not in models:
            raise ValueError(f"Algoritmo de clustering '{self.method}' no soportado. Los métodos disponibles son 'kmeans', 'density' y 'hierarchical'")
        return models[self.method]

    # Función de entrenamiento
    def fit(self, X, y=None):
        self.models = []
        self.labels = []

        for image in X:
            if image is None:
                self.models.append(None)
                self.labels.append(None)
                continue

            pixels = image if self.squashed else np.array(image).reshape(-1, 3)
            model = self._get_model()

            grid_search = GridSearchCV(model, self.param_grid, scoring=self.get_scoring_metric())
            grid_search.fit(pixels)
            best_model = grid_search.best_estimator_
            self.models.append(best_model)

            labels = best_model.predict(pixels) if hasattr(best_model, 'predict') else best_model.labels_
            self.labels.append(labels)

        return self

    # Transformación
    def transform(self, X):
        cluster_image_data = []
        cluster_results = []

        for i, image in enumerate(X):
            if image is None:
                cluster_image_data.append(None)
                cluster_results.append(None)
                continue

            pixels = image if self.squashed else np.array(image).reshape(-1, 3)
            model = self.models[i]

            if model is None:
                cluster_image_data.append(None)
                cluster_results.append(None)
                continue

            labels = model.predict(pixels) if hasattr(model, 'predict') else model.labels_
            label_class = np.unique(labels)

            cluster_colors = [pixels[labels == label].mean(axis=0) for label in label_class]
            cluster_results.append(np.array(cluster_colors))

            cluster_color_results = [(pixels[labels == label], np.where(labels == label)[0], cluster_colors[i]) 
                                     for i, label in enumerate(label_class)]
            cluster_image_data.append(cluster_color_results)

        return cluster_image_data, cluster_results

## Evaluación de los métodos aplicados a las imágenes mediante métricas

Iniciamos aplicando processed_images_array, el cual es el arreglo que contiene las imágenes preprocesadas, cada una de las cuales ha sido redimensionada y normalizada. Su forma original es (n_imágenes, altura, ancho, 3), donde:

- n_imágenes: El número de imágenes en el conjunto.
- altura y ancho: Las dimensiones de cada imagen.
- 
Recordemos que los tres canales de color son Rojo, Verde, Azul para cada imagen.

- reshape(processed_images_array.shape[0], -1):

Ahora aplicamos processed_images_array.shape[0] para mantener el número de imágenes (la primera dimensión del arreglo), es decir, no modifica la cantidad de imágenes.

In [ ]:
# Aplanar las imágenes procesadas para poder aplicar la métrica
data = processed_images_array.reshape(processed_images_array.shape[0], -1)

## Función `silhouette_plot`:

Esta función evalúa la calidad de los modelos de agrupación, como KMeans y AgglomerativeClustering, utilizando el Silhouette Score para medir la cohesión interna de los clusters. 
Los parámetros clave incluyen los datos a agrupar (`X`), el algoritmo a utilizar (`model`), y el rango de clusters (`k_min`, `k_max`). 
La función itera sobre diferentes valores de `k`, ajustando el modelo para cada uno, calculando el Silhouette Score que indica qué tan bien separados están los clusters. 
Si hay menos de dos clusters, asigna un puntaje de -1. 
Finalmente, grafica y guarda la variación del Silhouette Score en función del número de clusters, retornando la ruta de la imagen generada.

In [ ]:
# Métricas para la evaluación de los modelos
def silhouette_plot(X, model, k_min=2, k_max=10):
    if model not in ['kmeans', 'hierarchical']:
        raise ValueError(f"Modelo '{model}' no soportado para análisis de Silhouette Score")
    
    scores = []

    # Función auxiliar para seleccionar el modelo
    def get_model(k):
        if model == "kmeans":
            return KMeans(n_clusters=k, max_iter=300, random_state=0)
        elif model == "hierarchical":
            return AgglomerativeClustering(n_clusters=k)

    # Silhouette Score
    for k in range(k_min, k_max + 1):
        model_k = get_model(k)
        model_k.fit(X)

        # Solo se calcula el Silhouette Score si hay más de 1 cluster
        score = silhouette_score(X, model_k.labels_) if len(set(model_k.labels_)) > 1 else -1
        scores.append(score)

    # Gráfico del Silhouette Score
    plt.figure()
    plt.plot(range(k_min, k_max + 1), scores, marker='o')
    plt.xlabel('Número de clusters')
    plt.ylabel('Silhouette Score')
    plt.title(f'Evaluación de Silhouette Score usando {model}')
    plt.grid()

    save_path = f'silhouette_{model}.png'
    plt.savefig(save_path)
    plt.close()
    return save_path

A continuación, evaluamos el Silhouette Score para los modelos de agrupación KMeans y Clustering Jerárquico, variando el número de clusters entre 2 y 10. 
Para ambos algoritmos, se llama a la función silhouette_plot, que genera una gráfica con el Silhouette Score para cada número de clusters. Luego, las imágenes de las gráficas se cargan desde las rutas save_path_kmeans y save_path_hierarchical. Las gráficas se visualizan utilizando plt.imshow sin mostrar los ejes (axis('off')). Esto permite una comparación visual del rendimiento de los modelos en función de la calidad de sus agrupaciones, facilitando la elección del número óptimo de clusters para cada modelo.

In [ ]:
# KMeans (Clusters entre 2 y 10)
save_path_kmeans = silhouette_plot(data, "kmeans", 2, 10)

img = plt.imread(save_path_kmeans)
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

# Clustering Jerárquico (Clusters entre 2 y 10)
save_path_hierarchical = silhouette_plot(data, "hierarchical", 2, 10)

img = plt.imread(save_path_hierarchical)
plt.imshow(img)
plt.axis('off')
plt.tight_layout()
plt.show()

## Resultados

Para visualizar datos en 2D, se utilizarán métodos de reducción de dimensionalidad como PCA (lineal) y t-SNE (no lineal). Se implementará una clase que aplicará estos métodos y también graficará los resultados. La clase `DimensionReductor` permite reducir la dimensionalidad de imágenes y visualizarlas en 2D. Utiliza el algoritmo PCA para reducir los datos, devolviendo la varianza explicada por cada componente, mientras que con t-SNE transforma los datos directamente sin calcular varianza. Además, incluye métodos para procesar matrices de imágenes, reduciendo su dimensionalidad de forma individual para cada imagen. Para ayudar a entender la contribución de cada componente de PCA, la clase puede generar una gráfica que muestra la varianza explicada por cada uno. También permite visualizar los datos reducidos en 2D, ya sea coloreando los puntos según los clusters a los que pertenecen o sin considerar la agrupación, utilizando colores predefinidos. De esta manera, se facilita la interpretación de datos complejos mediante la reducción de dimensionalidad y su representación gráfica.grupación.

In [ ]:
# Constructor para visualizar los datos por medio de reduccion de dimensionalidad

class DimensionReductor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.reductor_model = None
        self.image_variances = []

    # Funciones de reducción de dimension PCA y TSNE
    def pca(self, X_image, n_components=2):
        pca_variance = PCA()
        pca = PCA(n_components=n_components)
        pca_variance.fit(X_image)
        X_image_redux = pca.fit_transform(X_image)
        variance = pca_variance.explained_variance_ratio_
        return variance, X_image_redux

    def tsne(self, X_image, n_components=2, perplexity=0.30, learning_rate=0.01, random_state=0, verbose=1):
        tsne = TSNE(n_components=n_components, perplexity=perplexity, learning_rate=learning_rate, random_state=random_state, verbose=verbose)
        X_image_redux = tsne.fit_transform(X_image)
        return X_image_redux

    def pca_images(self, X_image_matrix, n_components=2):
        X_reduced = []
        for image in X_image_matrix:
            image_variance, X_image_redux = self.pca(image, n_components=n_components)
            self.image_variances.append(image_variance)
            X_reduced.append(X_image_redux)
        return self.image_variances, X_reduced

    def tsne_images(self, X_image_matrix, n_components=2, perplexity=30.0, learning_rate='auto', random_state=0, verbose=0):
        X_reduced = []
        for image in X_image_matrix:
            X_image_redux = self.tsne(image, n_components=n_components, perplexity=perplexity,
                                      learning_rate=learning_rate, random_state=random_state, verbose=verbose)
            X_reduced.append(X_image_redux)
        return X_reduced

    # Plot de varianza para PCA
    def pca_variance_plot(self, image_variance):
        plt.figure(figsize=(8,4))
        plt.plot(image_variance)
        plt.xlabel('Varianza explicada por numero de componentes')
        plt.ylabel('Porcentaje de varianza')
        plt.title('Varianza explicada por componentes para la imagen')
        plt.grid(True)
        plt.show()

 # Scatter plot clusterizado y reducida dimensionalmente
    def redux_plot(self, used_method, X_reduced_image, cluster_indices, colors):
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))

        if len(cluster_indices) != len(colors):
            raise ValueError("cluster_indices y colors deben tener la misma longitud")

        for i, color in enumerate(colors):
            indices = cluster_indices[i]
            ax.scatter(X_reduced_image[indices, 0], X_reduced_image[indices, 1],
                      c=[color], label=f'Cluster {i}', alpha=0.6)

        ax.set_title(f'Reducción de dimensionalidad  {used_method}')
        plt.grid(True)
        plt.show()

    # Scatter plot no clusterizado y reducida dimensionalmente
    def redux_plot_non_clustered(self, used_method, X_reduced_image, colors):
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        ax.scatter(X_reduced_image[:, 0], X_reduced_image[:, 1], c=colors, alpha=0.6)
        ax.set_title(f'Reducción de dimensionalidad{used_method}')
        plt.grid(True)
        plt.show()

Cargaremos nuevamente las imágenes e implementaremos los modelos de clustering de color. Luego, compararemos los métodos de reducción de dimensionalidad mediante los siguientes gráficos: la paleta de colores de la imagen, la imagen original, y las proyecciones de los píxeles de color en dos dimensiones utilizando PCA y TSNE, tanto sin clusterización como con clusterización aplicada.

Para poder aplicar todos los procesos de procesamiento, clusterización y reducción de la dimensionalidad en una única secuencia organizada de transformación, generamos un pipeline que aplique cada uno de los procesos mencionados hasta este punto.

Por lo tanto, ahora procederemos a implementar una clase llamada `Color_Extractor`, que realiza la extracción y visualización de colores de imágenes utilizando un pipeline de procesamiento que incluye reducción de dimensionalidad y agrupación de colores. La clase hereda de varias otras clases como `Image_loader`, `Resizer`, `ColorConverter`, entre otras, para aplicar diferentes transformaciones y agrupaciones a las imágenes.

En su constructor, `Color_Extractor` inicializa varios parámetros, como el modelo de agrupación (por ejemplo, KMeans), el rango de componentes para la reducción de dimensionalidad, y las imágenes a procesar. El pipeline de preprocesamiento de imágenes incluye la redimensión, conversión de color, normalización y aplastamiento de las imágenes. La clase también puede aplicar técnicas de reducción de dimensionalidad como PCA o t-SNE antes o después del proceso de agrupación.

La clase utiliza un método para generar clusters, evaluando los datos y agrupándolos según el modelo de clustering seleccionado. Una vez que los clusters y la reducción de dimensionalidad han sido procesados, se grafican las paletas de colores obtenidas de las imágenes, mostrando visualmente los resultados en 2D, tanto con clusters como sin ellos. Además, ofrece la posibilidad de visualizar los datos originales y reducidos en espacios bidimensionadad.

In [ ]:
# Pipeline del procedimiento de extracción y visualización de color

class Color_Extractor(Image_loader, Resizer, ColorConverter, Normalizer, Squasher, ColorCluster, DimensionReductor, BaseEstimator, TransformerMixin):

    def __init__(self, path, image_indices, param_grid, scoring_metric='silhouette', dim_redux=(None, None), n_components=2, model='kmeans', **kwargs):

        self.model = model
        self.param_grid = param_grid
        self.scoring_metric = scoring_metric
        self.path  = path
        self.image_indices = image_indices
        self.dim_redux = dim_redux
        self.n_components = n_components
        self.kwargs = kwargs
        self.loader        = Image_loader(folder_path=self.path, image_indices = self.image_indices)
        self.images_paths  = [os.path.join(self.path, img) for img in os.listdir(self.path)]
        self.processed_images = self.image_processor(self.images_paths[self.image_indices[0]: self.image_indices[1]])
        self.X_images_redux   = None
        self.clustered_data   = None
        self.platettes        = None

    # Pipeline de preprocesamiento de los datos
    def image_processor(self, X):
        steps = [
        ('resize', Resizer(size=(128, 128))),
        ('color_convert', ColorConverter()),
        ('normalize', Normalizer()),
        ('squasher', Squasher())
        ]
        pipe = Pipeline(steps)
        return pipe.fit_transform(X)


    # Pipeline de reducción de dimensionalidad
    def dim_redux_processing(self, X):
        if self.dim_redux[0] == 'PCA':
            dim_reductor  = DimensionReductor()
            X_reducted = dim_reductor.pca_images(X, n_components=self.n_components)
        elif self.dim_redux[0] == 'TSNE':
            dim_reductor = DimensionReductor()
            X_reducted  = dim_reductor.tsne_images(X, n_components=self.n_components)
        return X_reducted


    # Generador de clusters
    def evaluate_clusters(self, X):
        if self.dim_redux[1] == 'prev':
            squashed = True
        else:
            squashed = False

        cluster_generator = ColorCluster(param_grid=self.param_grid, method=self.model, scoring_metric=self.scoring_metric, squashed=squashed,**self.kwargs)
        clustered_data, palettes = cluster_generator.fit_transform(X)
        return palettes, clustered_data


    # Pipeline de procesamiento con los modelos
    def ml_processing(self):
        if self.dim_redux[1] == 'prev':
            self.X_images_redux = self.dim_redux_processing(self.processed_images)

            if self.dim_redux[0] == 'PCA':
                explained_variances = self.X_images_redux[0]
                self.X_images_redux   = self.X_images_redux[1]
            palettes, clustered_data = self.evaluate_clusters(self.X_images_redux)

        elif self.dim_redux[1] == 'post':
            palettes, clustered_data = self.evaluate_clusters(self.processed_images)
            self.X_images_redux = self.dim_redux_processing(self.processed_images)

        self.clustered_data = clustered_data
        self.palettes = palettes


    # Paleta de colores
    def image_palette_plotter(self, palette):
        if np.max(palette) > 1:
            normalized_colors = palette / 255
        else:
            normalized_colors = palette
        plt.figure(figsize=(8, 2))
        for i, color in enumerate(normalized_colors):
            plt.fill_between([i, i+1], 0, 1, color=color)
        plt.xlim(0, len(normalized_colors))
        plt.axis('off')
        plt.show()


    # Visualización de la reducción de dimensionalidad
    def visualizer(self):
        self.ml_processing()
        order_redux = self.dim_redux[1]
        X_images = self.X_images_redux
        X_processed_images = self.processed_images
        palettes = self.palettes

        for i, image in enumerate(X_images):
            cluster_data = self.clustered_data[i]
            indices_clusters = [data[1] for data in cluster_data]

            if order_redux == 'post':
                colors = [data[2] for data in cluster_data]
                palette_ejemplo = palettes[i]
            elif order_redux == 'prev':
                colors_redux = [[X_processed_images[i,index] for index in clusters] for clusters in indices_clusters]
                palette_ejemplo = [np.mean(color, axis=0) for color in colors_redux]
                colors = palette_ejemplo

            # Graficación de la imagen y su paleta
            self.image_palette_plotter(palette_ejemplo)
            self.loader.plotter(index_image=i)

            # Graficación reducida 2D de la imagen en color no clusterizado
            original_data = X_processed_images[i]
            dim_reductor  = DimensionReductor()
            dim_reductor.redux_plot_non_clustered(used_method=self.dim_redux[0], X_reduced_image=image, colors=original_data)

            # Visualización de la reducción de la dimensioalidad de las imagenes
            flattened_indices = np.concatenate(indices_clusters)
            flattened_colors = np.concatenate(colors)
            dim_reductor.redux_plot(used_method=self.dim_redux[0], X_reduced_image=image, cluster_indices=indices_clusters, colors=colors)

Para seleccionar los clusters, los mejores valores obtenidos con el Silhouette Score están entre 5 y 7, excluyendo los dos primeros clusters que dieron los resultados más óptimos. Con estos valores, se creó un **param_grid** para evaluar las imágenes en diferentes modelos.

Ahora procedemos a crear una visualización comparativa de dos imágenes, una para el modelo KMeans y otra para el modelo Clustering Jerárquico, mostrando los resultados del análisis de Silhouette Score. Utiliza `plt.subplots` para generar dos subgráficos dispuestos en una fila con tamaño ajustado. 

Luego, cargamos las imágenes de las gráficas guardadas previamente en `save_path_kmeans` y `save_path_hierarchical`, mostrando cada una en su respectivo subgráfico.

In [ ]:
# Mostrar las imágenes una al lado de la otra
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Leer las imágenes y mostrarlas en los subplots
images = [save_path_kmeans, save_path_hierarchical]
titles = ["KMeans", "Hierarchical"]

for i, ax in enumerate(axes):
    img = plt.imread(images[i])
    ax.imshow(img)
    ax.set_title(titles[i])
    ax.axis('off')

# Añadir título general
plt.suptitle('Silhouette Score para n_clusters entre 2 y 10', fontsize=16)

plt.tight_layout()
plt.show()

Tras discutir cómo funciona el método de extracción de la paleta de colores, generamos resultados para una muestra aleatoria, donde el número de colores se determina automáticamente mediante Grid Search. Cada resultado incluye los siguientes gráficos: la paleta de colores, la imagen original, los píxeles de color en un contexto de dimensión reducida y los píxeles de color en dimensión reducida con clusterización aplicada.

Ahorta bien, debemos implementar un modelo KMeans con reducción de dimensionalidad mediante PCA antes de aplicar el agrupamiento. El proceso comienza midiendo el tiempo de ejecución con `time.time()` con la finalidad de terminar el tiempo que tarda en ejecutar el algoritmo y así tener un parámetro de referencia para considerar que un modelo es mejor o no que otro. 

Luego, se especifica una carpeta de imágenes y se define un rango de imágenes para procesar. Luego, se crea un pipeline con el objeto `Color_Extractor`, que aplica PCA para reducir la dimensionalidad a 2 componentes antes de ejecutar KMeans con una búsqueda de parámetros (`n_clusters` y `init`). El modelo se ajusta utilizando el Silhouette Score para evaluar la calidad de los clusters. 

Finalmente, se llama al método `visualizer` para mostrar los resultados de la agrupación y se imprime el tiempo total de ejecución del proceso.

In [ ]:
# Modelo KMEANS con PCA

start_time = time.time()

image_folder = "imagenes"
indices_imagenes = (0, 1)  # Usar una imagen (de la primera a la segunda imagen)


image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]

# Prueba del modelo kmeans con PCA
param_grid = {
    'n_clusters': [5,6,7],
    'init': ['k-means++', 'random']
}

color_extractor = Color_Extractor(image_folder , indices_imagenes , param_grid=param_grid, scoring_metric='silhouette',
                                  dim_redux=('PCA', 'prev'), n_components=2, model='kmeans')
color_extractor.visualizer()

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")

Ahora procederemos a implementar un modelo KMeans con reducción de dimensionalidad mediante t-SNE antes de aplicar el agrupamiento. Se inicia estableciendo un rango de imágenes para procesar, y el modelo KMeans se ajusta con diferentes valores de `n_clusters` y métodos de inicialización. El pipeline de preprocesamiento utiliza t-SNE para reducir la dimensionalidad de las imágenes a 2 componentes antes de realizar la agrupación. 
El proceso es gestionado por la clase `Color_Extractor`, que incluye la visualización de los resultados mediante el método `visualizer()`. 
Finalmente, se mide y muestra el tiempo de ejecución total del proceso, desde el inicio hasta el final.

In [ ]:
# KMEANS CON TSNE

start_time = time.time()

indices_imagenes = (0, 1)  # Usar una imagen (de la primera a la segunda imagen)


param_grid = {
    'n_clusters': [5,6,7],
    'init': ['k-means++', 'random']
}

color_extractor = Color_Extractor(image_folder , indices_imagenes , param_grid=param_grid , dim_redux=('TSNE', 'prev'), n_components=2, model='kmeans')
color_extractor.visualizer()

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")

Ahora procederemos a implementar el modelo DBSCAN con reducción de dimensionalidad mediante PCA antes del agrupamiento. 
Se define un conjunto de parámetros, incluyendo `eps`, `min_samples`, y `metric`, para ajustar el modelo DBSCAN a través de una búsqueda en el grid. 
El pipeline de preprocesamiento aplica PCA para reducir la dimensionalidad de las imágenes a 2 componentes antes de realizar la agrupación. El proceso es gestionado por la clase `Color_Extractor`, que combina la reducción de dimensionalidad y el modelo DBSCAN. Posteriormente, se visualizan los resultados utilizando el método `visualizer()`.

In [ ]:
#DBSCAN con PCA

start_time = time.time()

indices_imagenes = (0, 1)  # Usar una imagen (de la primera a la segunda imagen)

param_grid = {
    'eps': [0.01, 0.1, 1],
    'min_samples': [5, 10, 20],
    'metric': ['euclidean', 'manhattan']
}

color_extractor = Color_Extractor(image_folder , indices_imagenes , param_grid=param_grid , dim_redux=('PCA', 'prev'), n_components=2, model='density')
color_extractor.visualizer()

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")

Ahora aplicaremos el modelo DBSCAN con reducción de dimensionalidad utilizando t-SNE antes de realizar el agrupamiento. Se definen los parámetros para el modelo DBSCAN, como `eps`, `min_samples` y `metric`, los cuales se ajustan mediante una búsqueda de hiperparámetros. El pipeline de preprocesamiento utiliza t-SNE para reducir la dimensionalidad de las imágenes a 2 componentes antes de la agrupación. La clase `Color_Extractor` gestiona todo el proceso y visualiza los resultados a través del método `visualizer()`. Finalmente, el código mide el tiempo total de ejecución del proceso y lo imprime, lo que permite evaluar tanto la eficiencia como el rendimiento del modelo DBSCAN combinado con la reducción de dimensionalidad mediante t-SNE.

In [ ]:
#DBSCAN con TSNE 
start_time = time.time()

param_grid = {
    'eps': [0.01, 0.1, 1],
    'min_samples': [5, 10, 20],
    'metric': ['euclidean', 'manhattan']
}

indices_imagenes = (0, 1)  # Usar una imagen (de la primera a la segunda imagen)

color_extractor = Color_Extractor(image_folder , indices_imagenes , param_grid=param_grid , dim_redux=('TSNE', 'prev'), n_components=2, model='density')
color_extractor.visualizer()

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")

Ahora, lo que haremos será implementar un modelo de agrupamiento jerárquico utilizando reducción de dimensionalidad con PCA. 
Para esto, se define un conjunto de parámetros (`n_clusters`, `linkage` y `affinity`) que se ajustan mediante una búsqueda de hiperparámetros. El proceso aplica PCA para reducir la dimensionalidad de las imágenes a 2 componentes antes de realizar la agrupación jerárquica. La clase `Color_Extractor` gestiona tanto la reducción de dimensionalidad como la ejecución del modelo, visualizando los resultados a través de su método `visualizer()`.

In [ ]:
#Modelo Jerárquico con PCA
start_time = time.time()

param_grid_agglomerative = {
    'n_clusters': [5, 6, 7],
    'linkage': ['ward', 'complete', 'average', 'single'],
    'affinity': ['euclidean', 'manhattan']
}

indices_imagenes = (0, 1)  # Usar una imagen (de la primera a la segunda imagen)


color_extractor = Color_Extractor(image_folder , indices_imagenes , param_grid=param_grid_agglomerative , dim_redux=('PCA', 'prev'), n_components=2, model='hierarchical')
color_extractor.visualizer()

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")

Finalmente, implementamos un modelo de agrupamiento jerárquico con reducción de dimensionalidad mediante t-SNE antes de aplicar el agrupamiento. Se define un conjunto de parámetros (`n_clusters`, `linkage` y `affinity`) para ajustar el modelo de agrupamiento jerárquico. La reducción de dimensionalidad a 2 componentes se realiza utilizando t-SNE, y el proceso completo es manejado por la clase `Color_Extractor`. El método `visualizer()` se utiliza para mostrar los resultados de la agrupación y la reducción de dimensionalidad.

In [ ]:
#Modelo Jerárquico con TSNE
start_time = time.time()

param_grid_agglomerative = {
    'n_clusters': [5, 6, 7],
    'linkage': ['ward', 'complete', 'average', 'single'],
    'affinity': ['euclidean', 'manhattan']
}
indices_imagenes = (0, 1)  # Usar una imagen (de la primera a la segunda imagen)


color_extractor = Color_Extractor(image_folder , indices_imagenes , param_grid=param_grid_agglomerative, dim_redux=('TSNE', 'prev'), n_components=2, model='hierarchical')
color_extractor.visualizer()

end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time} segundos")

## Conclusiones

De acuerdo a las ejecuciones hechas en este notebook, es posible concluir que el mejor modelo ejecutado fue el KMEANS con PCA debido a que mostró un mejor perfil de color de acuerdo a la imagen procesada, permitiendo verificar una efectiva reducción de la dimensionalidad a partir de un amplio espectro de colores.
En cuanto a la dimensión de la imagen, para este caso escogimos (128,128) pixeles dado que se considera equilibrada en términos de resolución, información y optimización de tiempos de ejecución de los algoritmos. Si bien se pudo escoger una dimensión (256,256) se considera que este tamaño complejiza los modelos y a la hora de aplicar las respectivas funciones de extraccion de paleta. 
Por otro lado, podemos hablar de optimizar los algoritmos reduciendo las funcionalidades del presente trabajo y considerando solo aquellas que hagan un debido procesamiento de la imagen, manejando las transformaciones de manera más óptima, pero consideramos que estas son las funciones mínimas que se deben implementar para no perder información en el procesamiento de las imagenes, transformación de información y aplicación de los algoritmos no supervisados. 
En cuanto al modelo DBSCAN con PCA, este podria considerarse a la hora de definir una paleta a decisión del usuario, ya que no se limita a unos colores estándar específicos sino que muestra la variación de izquierda a derecha de los colores. Si bien el tiempo de ejecución no es el menor, computacionalmente hablando es un algoritmo efectivo en comparación con el modelo jerárquico con TSNE. 
Ahora bien, si consideramos la reducción de dimensionalidad del DBSCAN con PCA, observamos que logramos discernir clusters de manera definida, a pesar de que se solapan debido muy posiblemente a la resolución de la imagen.
Sin embargo, el modelo más óptimo sin duda fue el KMEANS con PCA debido a su tiempo de ejecución, su rendimiento a la hora de ejecutar y la posterior visualización de los clusters esperados, lo cual indicó que se hizo una efectiva reducción de la dimensionalidad.

### Método escogido aplicado a 4 imágenes aleatorias:

### KMEANS con PCA

In [ ]:
# KMEANS CON PCA
param_grid = {
    'n_clusters': [5,6,7],
    'init': ['k-means++', 'random']
}

color_extractor = Color_Extractor(image_folder , tuple(random.sample(range(100), 4)) , param_grid=param_grid, scoring_metric='silhouette',
                                  dim_redux=('PCA', 'prev'), n_components=2, model='kmeans')
color_extractor.visualizer()